In [1]:
import numpy as np
import matplotlib
from mido import Message, MetaMessage
import mido
import sys
import random
import os
import time
import math
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device="cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
pwd = "models/"
savepwd="/home/zedroman/Documents/Sonia_Data/Songs/_test2/"
max_len = 1000
models = [f for f in os.listdir(pwd) if f[len(f)-8:len(f)] != "_classes"]
dudets = False

cuda


In [2]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, other):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru1 = nn.GRU(classes, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)
        self.gru3 = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

        self.velocity = other[0]
        self.time_range = other[1]
        self.tempo = other[2]
        self.pedal_max = other[3]
        self.notes = other[4]

    def forward(self, input, hidden):
        output, hidden[0] = self.gru1(input, hidden[0])
        output = F.relu(output)
        output, hidden[1] = self.gru2(output, hidden[1])
        output = F.relu(output)
        output, hidden[2] = self.gru3(output, hidden[2])
        output = self.softmax(self.out(output[0]))
        
        return output, hidden
    
def list_find(l, n):
    for i in range(len(l)):
        if l[i] == n:
            return i
    return -1

def msg_to_class(msg):
    note = use_notes[msg.note]
    t = list_find(time_range, msg.time)
    v = list_find(velocity_range, msg.velocity)
    return v * len(use_notes) * len(time_range) + t * len(use_notes) + note
    
def class_to_msg(n):
    note = class_notes[n % len(class_notes)]
    n //= len(class_notes)
    t = time_range[n % 3]
    n //= 3
    v = velocity_range[n]
    return Message("note_on", note = note, velocity = v, time= t)


def midi_to_list(mid=None, pwd=None, tensor=True):
    if(mid==None):
          mid = mido.MidiFile(pwd)
    sample = []
    for i, track in enumerate(mid.tracks):
        for msg in track:
            if (msg.type == "note_on"):
                sample.append(msg_to_class(msg))
    if (tensor):
        return torch.tensor(sample, device=device)
    else:
        return sample

def list_to_midi(sample):
    midi = mido.MidiFile()
    tr = mido.MidiTrack()
    midi.tracks.append(tr)
    tr.append(MetaMessage("set_tempo", tempo=tempo, time=0))
    
    tr.append(MetaMessage("key_signature", key='A', time=0))
    
    pedal = pedal_max
    last_notes = []
    
    for i in sample:
        if (use_pedal):
            if (pedal == 0):
                for j in range(len(last_notes) - 1):
                    tr.append(Message("note_off", note=last_notes[j], velocity=0, time=0, channel=0))
                pedal = pedal_max
                last_notes = [last_notes[-1]]
        tr.append(class_to_msg(i))
        if (use_pedal):
            pedal -= 1
            last_notes.append(class_note[i % len(class_note)])
    if (use_pedal):
        for i in last_notes:
            tr.append(Message("note_off", note=i, velocity=0, time=0, channel=0))
    tr.append(MetaMessage("end_of_track", time = 0))
    return midi

In [48]:
def one_hot(c):
    x = torch.zeros(classes, device=device, dtype=torch.float)
    x[c] = 1;
    return x.view(1,1,-1)

def evaluate(decoder,num=-1, max_length=max_len):
    with torch.no_grad():

        decoder_input = one_hot(sos)
        decoder_hidden = []
        for j in range(3):
            decoder_hidden.append([random.random() * 10 for i in range(hidden_size)])
        decoder_hidden =   [torch.tensor(decoder_hidden[0], device=device).view(1,1,-1),
                        torch.tensor(decoder_hidden[1], device=device).view(1,1,-1),
                        torch.tensor(decoder_hidden[2], device=device).view(1,1,-1)]

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(3)
            topv = (-1)/topv.cpu()[0].numpy()
            summ = topv.sum()
            chosen = np.random.choice(topi[0].cpu(), p= topv / summ)
            decoder_input = one_hot(chosen)
            decoded_words.append(chosen)

        return decoded_words

def create_dudec(length):
    tr = mido.MidiTrack()
    d = mido.MidiFile('Dudets.mid')
    messages = []
    for track in d.tracks:
        for msg in track:
            if (msg.type == 'note_off' or msg.type == 'note_on'):
                msg.time = int(msg.time * 1.5)
                msg.velocity = 50
                messages.append(msg)
    for i in range(int(length) // int(d.length)):
        for i in messages:
            tr.append(i)
    return tr
    
def generate_sample(number, decoder):
    for i in range(number):
        new_music = evaluate(decoder, max_length=1000)
        midi = list_to_midi(new_music)
        print(midi.length)
        if (dudets):
            midi.tracks.append(create_dudec(midi.length))
        midi.save(savepwd +"sample" + str(i) + ".mid")
    return midi

In [49]:
def generate(model):
    with open(pwd + model + '_classes') as f: 
        file = f.readlines()[0].split()
        global classes, tempo, time_range, velocity_range, pedal_max, class_notes, sos, hidden_size, use_pedal
        classes = int(file[0])
        hidden_size = int(file[1])
    sos = classes - 1
    use_pedal = True
    decoder = DecoderRNN(hidden_size, classes, [0,0,0,0,0]).to(device)
    checkpoint = torch.load(pwd+model)
    class_notes = checkpoint['class_note']
    pedal_max = checkpoint['pedal_max']
    velocity_range = checkpoint['velocity_range']
    time_range = checkpoint['time_range']
    tempo = checkpoint['tempo']
    decoder.load_state_dict(checkpoint['model_state_dict'])
    generate_sample(1, decoder)
    

In [50]:
def main():
    print("Choose Model\n type it's number")
    for num, model in enumerate(models):
        print(num, model)
    model = int(input())
    if (model >= len(models)):
        print("fuck you!")
        return
    generate(models[model])

In [51]:
main()

Choose Model
 type it's number
0 anime
1 bethoven
2 betoven_lvl2
0
[0.99543375 0.5652396  0.5108578 ]

[17.695992    0.28822047  0.21098894]

[6.2360673  0.38002348 0.2753205 ]

[1.0474594  0.8929158  0.64611113]

[0.7396892 0.5707308 0.5515977]

[2.340663   0.43373328 0.36214343]

[0.7569842  0.52833116 0.47616184]

[1.3650498  0.42026025 0.35265514]

[1.1226351 0.9069856 0.5504642]

[3.7932787  0.48011953 0.243649  ]

[2.1376433  0.3948574  0.38673398]

[0.8788219  0.45760888 0.42828003]

[6.2507696  0.28832856 0.25444826]

[15.959818    0.30565703  0.2307762 ]

[1.1087337  0.9043865  0.41789895]

[1.0731144  0.72454673 0.5136725 ]

[0.8481919  0.7643002  0.70515996]

[3.5347245  0.5531395  0.29946142]

[0.85333437 0.54468155 0.4457514 ]

[1.1654645  0.56162137 0.3837149 ]

[0.8984207  0.51177484 0.50145596]

[1.5223804  1.1744237  0.25929368]

[81.27866     0.17703389  0.17409937]

[6.8177893e+02 1.2651567e-01 1.2072280e-01]

[0.5805921  0.49470022 0.41870716]

[9.819598   0.2924294

[1.1721079  0.9767752  0.44486228]

[13.8488035   0.25802964  0.21570958]

[8.694877   0.32149574 0.2370609 ]

[1.757012   0.49867672 0.418698  ]

[2.6461952  0.48679957 0.38322654]

[3.96647    0.58256876 0.23355336]

[76.04438     0.18370801  0.18151042]

[9.78149231e+02 1.33728161e-01 1.09532654e-01]

[108.27922      0.20459312   0.1312028 ]

[14.484294    0.28485793  0.21954344]

[113.55599      0.15414122   0.1418752 ]

[1.9616547 0.5611034 0.3823275]

[13.28438     0.28759697  0.21917361]

[13.486682    0.319798    0.25161588]

[38.465736    0.22434625  0.17322735]

[0.9566313  0.84211034 0.42087924]

[2.0669742e+02 1.7336999e-01 1.2188498e-01]

[16.415806    0.24928658  0.24460842]

[2.5955172  0.56020766 0.46727714]

[0.96428406 0.76633376 0.6868724 ]

[1.4361242  1.1038804  0.29476994]

[3.7879071  0.36178696 0.30830225]

[3.9354608  0.32153133 0.27196765]

[2.4248064  0.8773872  0.18542767]

[1.0696819 0.461907  0.3858393]

[1.0043403  0.95974356 0.34000525]

[1.0107926  0.94

[2.64824    0.7582525  0.26280877]

[17.408663    0.27794975  0.20585567]

[4.4712534  0.593477   0.18334888]

[25.221918    0.25016302  0.20740592]

[7.271981   0.34989628 0.23987313]

[2.4533832e+02 1.6471587e-01 1.4372304e-01]

[14.929962    0.24318863  0.23708291]

[1.1553475  0.97437084 0.5316465 ]

[3.290156  0.6138083 0.3097234]

[86.982666    0.174441    0.17368601]

[1.5903105 1.0253468 0.2889784]

[3.5405846  0.57116526 0.23514184]

[7.522174   0.4134749  0.20088942]

[6.4967535e+02 1.3037147e-01 1.2761834e-01]

[11.550611    0.3708922   0.17861259]

[1.1727976 0.9415991 0.3517441]

[1.9285362  0.88024056 0.37948188]

[103.21646      0.19396067   0.15682825]

[2.4137766  0.82735986 0.23414871]

[18.28699     0.3007019   0.22115502]

[28.018063    0.22605315  0.20029755]

[4.607242   0.42372495 0.30566236]

[24.516624   0.2724652  0.1777164]

[31.88712     0.22326607  0.19411793]

[67.17765     0.21783015  0.15632613]

[9.360698   0.26787758 0.2609978 ]

[3.4210062  0.52701557

NameError: name 'class_note' is not defined

In [159]:
print(dudec)

<midi file 'Dudets.mid' type 1, 2 tracks, 21 messages>


In [82]:
dudec = mido.MidiFile('Dudets.mid')
for track in dudec.tracks:
    print(track)
    for msg in track:
        print(msg)

<midi track '' 3 messages>
<meta message set_tempo tempo=461538 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message end_of_track time=0>
<midi track 'Serum 1' 18 messages>
<meta message track_name name='Serum 1' time=0>
note_on channel=0 note=89 velocity=100 time=336
note_off channel=0 note=89 velocity=64 time=48
note_on channel=0 note=88 velocity=100 time=0
note_off channel=0 note=88 velocity=64 time=144
note_on channel=0 note=89 velocity=100 time=0
note_off channel=0 note=89 velocity=64 time=48
note_on channel=0 note=85 velocity=100 time=0
note_off channel=0 note=85 velocity=64 time=144
note_on channel=0 note=85 velocity=100 time=0
note_off channel=0 note=85 velocity=64 time=48
note_on channel=0 note=84 velocity=100 time=0
note_off channel=0 note=84 velocity=64 time=144
note_on channel=0 note=85 velocity=100 time=0
note_off channel=0 note=85 velocity=64 time=48
note_on channel=0 note=82 velocity=100 ti

In [23]:
for i in range(10):
    print(random.randint(1,3))

1
2
2
2
3
1
3
3
1
2
